# Data Visualizeation

The purpose of this notebopok is to learn how to clean and visualize data

Some tools/libraries being used are:
- **Pandas**
- **Seaborn**
- **Matplotlib**
- **Numpy**

Before Visualizing we should do some cleaning first \
(i.e.) Removing/imputing NaN values, encoding string values, etc


In [24]:
import pandas as pd
import regex as re
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.image import thumbnail

In [25]:
lego_df = pd.read_csv('lego_sets.csv')

In [26]:
lego_df.head()

,set_id,name,year,theme,subtheme,themeGroup,category,pieces,minifigs,agerange_min,US_retailPrice,bricksetURL,thumbnailURL,imageURL
0,1-8,Small house set,1970,Minitalia,NaN,Vintage,Normal,67.0,NaN,NaN,NaN,https://brickset.com/sets/1-8,https://images.brickset.com/sets/small/1-8.jpg,https://images.brickset.com/sets/images/1-8.jpg
1,2-8,Medium house set,1970,Minitalia,NaN,Vintage,Normal,109.0,NaN,NaN,NaN,https://brickset.com/sets/2-8,https://images.brickset.com/sets/small/2-8.jpg,https://images.brickset.com/sets/images/2-8.jpg
2,3-6,Medium house set,1970,Minitalia,NaN,Vintage,Normal,158.0,NaN,NaN,NaN,https://brickset.com/sets/3-6,https://images.brickset.com/sets/small/3-6.jpg,https://images.brickset.com/sets/images/3-6.jpg
3,4-4,Large house set,1970,Minitalia,NaN,Vintage,Normal,233.0,NaN,NaN,NaN,https://brickset.com/sets/4-4,https://images.brickset.com/sets/small/4-4.jpg,https://images.brickset.com/sets/images/4-4.jpg
4,4-6,Mini House and Vehicles,1970,Samsonite,Model Maker,Vintage,Normal,NaN,NaN,NaN,NaN,https://brickset.com/sets/4-6,NaN,NaN


## Describing the dataset

### 1) Finding duplicate values

Are there any duplicates in the dataset? If so gotta get rid of them\
They are going to be dropped

### 2) Find the data types of each column

This will help us figure what data types are in each column so they can be changed or left alone \
Typically with string values they should be encoded/left alone/dropped

### 3) Find the Null Values

After handling data types we need to handle and NaN values
We can either drop or impute, once again it is a case by case situation


-----

So now have a look at these tabels
- DTYPES: Type of data in each column
- MISSING: What columns have missing values
- PERCENTAGE MISSING: The percentage missing values in each column


In [27]:
# find duplicate rows
lego_df[lego_df.duplicated(['name','set_id'], keep=False)]
# there are none


,set_id,name,year,theme,subtheme,themeGroup,category,pieces,minifigs,agerange_min,US_retailPrice,bricksetURL,thumbnailURL,imageURL


In [28]:
print("Lego data from Data types")
print(lego_df.dtypes)
print("="*28 + "\n")
print("Lego null values sum")
print(lego_df.isnull().sum())
print("="*28 + "\n")
print("Lego missing percentage")
print(lego_df.isnull().sum()/lego_df.isnull().count() * 100)
print("="*28 + "\n")


Lego data from Data types
set_id             object
name               object
year                int64
theme              object
subtheme           object
themeGroup         object
category           object
pieces            float64
minifigs          float64
agerange_min      float64
US_retailPrice    float64
bricksetURL        object
thumbnailURL       object
imageURL           object
dtype: object

Lego null values sum
set_id                0
name                  0
year                  0
theme                 0
subtheme           3556
themeGroup            2
category              0
pieces             3924
minifigs          10058
agerange_min      11670
US_retailPrice    11475
bricksetURL           0
thumbnailURL       1006
imageURL           1006
dtype: int64

Lego missing percentage
set_id             0.000000
name               0.000000
year               0.000000
theme              0.000000
subtheme          19.266403
themeGroup         0.010836
category           0.000000
piec

After looking there are some things to note: \
Starting with dtypes:
| Column | dtype |
| ----------- | ----------- |
| set_id | object |
| name | object |
| theme | object |
| subtheme | object |
| themeGroup | object |
| category | object |
| bricksetURL  | object |
| thumbnailURL | object |
| imageURL | object |

From the looks of it most of the data in this set are object data types\

Also two columns come out with > 20% in missing values and those are: **pieces**(21.26%), **minifigs**(54.46%), **agerange_min**(63.22%), and **US_retailPrice**(62.17%)

These can definitely be webscrapped, for that a webscapper is will be necessary but that will be handled later \
NOTE: The subtheme columns has 19% missing values, we can just drop the rows with that those missing values

So
1) Save URLs for webscrapping
2) Drop the columns that aren't theme, subtheme and category
3) Drop rows with missing subtheme values
3) Convert float64 to int64
3) Take a look at the unique values of theme, subtheme, and category


In [29]:
# 1)
# save the URLs to a df
lego_df_urls = lego_df[["bricksetURL", "imageURL", "thumbnailURL", "set_id"]]
print(lego_df_urls.head())

# 2)
# drop all columns of dtype obj that isn't in this list
cols_to_keep = ["theme", "subtheme", "category", "set_id"]   # spelling must match your frame
lego_df = lego_df.drop(
    columns=[c for c in lego_df.select_dtypes("object").columns
             if c not in cols_to_keep]
)
# 3)
# drop the rows with NaN subtheme values
lego_df = (
    lego_df                # start with current frame
      .dropna(subset=["subtheme"])                 # 1) remove NaN in subtheme
      .loc[lambda d: d["subtheme"].str.strip() != ""]   # 2) remove empty strings
)

# 4)
# convert float64 to int64
int_cols = ["year", "pieces", "minifigs"]
lego_df[int_cols] = lego_df[int_cols].astype("Int64")


                     bricksetURL  \
0  https://brickset.com/sets/1-8   
1  https://brickset.com/sets/2-8   
2  https://brickset.com/sets/3-6   
3  https://brickset.com/sets/4-4   
4  https://brickset.com/sets/4-6   

                                          imageURL  \
0  https://images.brickset.com/sets/images/1-8.jpg   
1  https://images.brickset.com/sets/images/2-8.jpg   
2  https://images.brickset.com/sets/images/3-6.jpg   
3  https://images.brickset.com/sets/images/4-4.jpg   
4                                              NaN   

                                     thumbnailURL set_id  
0  https://images.brickset.com/sets/small/1-8.jpg    1-8  
1  https://images.brickset.com/sets/small/2-8.jpg    2-8  
2  https://images.brickset.com/sets/small/3-6.jpg    3-6  
3  https://images.brickset.com/sets/small/4-4.jpg    4-4  
4                                             NaN    4-6  


In [30]:
# 4) look at unique values and counts
print("Lego null values sum")
print(lego_df.isnull().sum())
print("="*28 + "\n")
print("Lego missing percentage")
print(lego_df.isnull().sum()/lego_df.isnull().count() * 100)
print("="*28 + "\n")

Lego null values sum
set_id               0
year                 0
theme                0
subtheme             0
category             0
pieces            3785
minifigs          7991
agerange_min      9004
US_retailPrice    8834
dtype: int64

Lego missing percentage
set_id             0.000000
year               0.000000
theme              0.000000
subtheme           0.000000
category           0.000000
pieces            25.400980
minifigs          53.627273
agerange_min      60.425475
US_retailPrice    59.284612
dtype: float64



Now we take a look at the unique values and the number of unique values

In [31]:
print("Theme unique values")
print(lego_df["theme"].unique())
print(lego_df["theme"].nunique())
print("="*50 + "\n")
print("Subtheme unique values")
print(lego_df["subtheme"].unique())
print(lego_df["subtheme"].nunique())
print("="*50 + "\n")
print("Category unique values")
print(lego_df["category"].unique())
print(lego_df["category"].nunique())

Theme unique values
['Samsonite' 'Trains' 'Books' 'LEGOLAND' 'Duplo' 'Universal Building Set'
 'System' 'Gear' 'Basic' 'Dacta' 'Technic' 'Service Packs' 'Promotional'
 'Town' 'Castle' 'Space' 'Scala' 'Pirates' 'Assorted' 'Creator' 'Aquazone'
 'Western' 'Education' 'Time Cruisers' 'Adventurers' 'Mindstorms'
 'Seasonal' 'Rock Raiders' 'Belville' 'Star Wars' 'Baby' 'Sports'
 'Advanced models' 'Bulk Bricks' 'Racers' 'Studios' 'Bionicle'
 'Harry Potter' 'Explore' 'Alpha Team' 'Miscellaneous' 'World City'
 'Island Xtreme Stunts' '4 Juniors' 'Spider-Man' 'Clikits' 'City' 'Batman'
 'Exo-Force' 'Factory' 'Make and Create' 'Indiana Jones' 'Power Functions'
 'Architecture' 'Bricks and More' 'Games' 'Power Miners' 'HERO Factory'
 'Prince of Persia' 'Toy Story' 'Collectable Minifigures' 'Atlantis'
 'Ninjago' 'Pirates of the Caribbean' 'Cars' 'Ideas' "Pharaoh's Quest"
 'DC Comics Super Heroes' 'Friends' 'Marvel Super Heroes'
 'The Lord of the Rings' 'Monster Fighters' 'The Hobbit'
 'Teenage Mutant N

### Filling null values

As seen, the last columns have quite a few missing values \
Sometimes they can be filled in with medians, means, etc, but we are going to try to actually find them with a webscrapper

Im sure the data for pieces, minifigs, and the price can be found, the ones that cannot will be filled in

In [32]:
needs_fill = lego_df.loc[
    lego_df["pieces"].isna() |
    lego_df["minifigs"].isna() |
    lego_df["US_retailPrice"].isna(),
    ["set_id"]                              # Brickset & Rebrickable key
].drop_duplicates()


needs_fill.head()



,set_id
4,4-6
5,078-1
6,104-1
7,126-1
8,157-3


In [33]:
lego_df_urls.to_csv("lego_urls_setIDs.csv", index=True)

